In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
from rnacomp.fastai_fit import *
from rnacomp.dataset import RNA_DatasetV1, LenMatchBatchSampler, DeviceDataLoader
from rnacomp.models import GraphTransformer
import gc

import torch
from x_transformers import TransformerWrapper, Decoder, Encoder


class CustomTransformer(nn.Module):
    def __init__(self,dim=192, depth=12,  head_size=32):
        super().__init__()
        self.dec = TransformerWrapper(
            num_tokens = 4,
            logits_dim = 2, 
            max_seq_len = 512,
            attn_layers = Decoder(
                dim = dim,
                depth = depth,
                heads = dim//head_size,
                attn_flash = True, 
                rotary_pos_emb = True
            )
         )
        
    def forward(self, x0):
        mask = x0['mask']
        Lmax = mask.sum(-1).max()
        mask = mask[:,:Lmax]
        x = x0['seq'][:,:Lmax]
        x = self.dec(x, mask = mask)
        return x
    
class SinusoidalPosEmb(nn.Module):
    def __init__(self, dim=16, M=10000):
        super().__init__()
        self.dim = dim
        self.M = M

    def forward(self, x):
        device = x.device
        half_dim = self.dim // 2
        emb = math.log(self.M) / half_dim
        emb = torch.exp(torch.arange(half_dim, device=device) * (-emb))
        emb = x[...,None] * emb[None,...]
        emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
        return emb

class RNA_Model(nn.Module):
    def __init__(self, dim=192, depth=12, head_size=32, **kwargs):
        super().__init__()
        self.emb = nn.Embedding(4,dim)
        self.pos_enc = SinusoidalPosEmb(dim)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=dim//head_size, dim_feedforward=4*dim,
                dropout=0.1, activation=nn.GELU(), batch_first=True, norm_first=True), depth)
        self.proj_out = nn.Linear(dim,2)
    
    def forward(self, x0):
        mask = x0['mask']
        Lmax = mask.sum(-1).max()
        mask = mask[:,:Lmax]
        x = x0['seq'][:,:Lmax]
        
        pos = torch.arange(Lmax, device=x.device).unsqueeze(0)
        pos = self.pos_enc(pos)
        x = self.emb(x)
        x = x + pos
        
        x = self.transformer(x, src_key_padding_mask=~mask)
        x = self.proj_out(x)
        
        return x

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    
def loss(pred,target):
    p = pred[target['mask'][:,:pred.shape[1]]]
    y = target['react'][target['mask']].clip(0,1)
    loss = F.l1_loss(p, y, reduction='none')
    loss = loss[~torch.isnan(loss)].mean()
    
    return loss

def loss_unc(pred,target):
    p = pred[target['mask'][:,:pred.shape[1]]]
    y = target['react'][target['mask']]#.clip(0,1)
    loss = F.l1_loss(p, y, reduction='none')
    loss = loss[~torch.isnan(loss)].mean()
    
    return loss

class MAE(Metric):
    def __init__(self): 
        self.reset()
        
    def reset(self): 
        self.x,self.y = [],[]
        
    def accumulate(self, learn):
        x = learn.pred[learn.y['mask'][:,:learn.pred.shape[1]]].clip(0,1)
        y = learn.y['react'][learn.y['mask']].clip(0,1)
        self.x.append(x)
        self.y.append(y)

    @property
    def value(self):
        x,y = torch.cat(self.x,0),torch.cat(self.y,0)
        loss = F.l1_loss(x, y, reduction='none')
        loss = loss[~torch.isnan(loss)].mean()
        return loss
    
def loss_laplace(pred,target):
    p = pred[target['mask'][:,:pred.shape[1]]].float()
    y = target['react'][target['mask']].clip(0,1).float()
    y_err = target['react_err'][target['mask']].float()
    
    loss = F.l1_loss(p, y, reduction='none')
    m = ~torch.isnan(loss)
    loss = (loss[m]/torch.sqrt(1.0+torch.nan_to_num(y_err[m],100.0).clip(0,100.0))).mean()
    
    return loss


In [4]:
class CFG:
    path = Path("../data/split")
    pathbb = Path("../data/Ribonanza_bpp_files")
    split_id = 'v0'
    bs = 1024
    num_workers = 12
    device = 'cuda'
    adjnact_prob = 0.5
    seed = 2023
    out = 'exp_05_lb_sam'
    
    dim = 192
    depth = 12
    dim_head = 32

seed_everything(CFG.seed)
os.makedirs(CFG.out, exist_ok=True)

In [5]:
fns = list(CFG.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
df_train = pd.read_parquet(CFG.path / f"train_data_{CFG.split_id}.parquet")
df_valid = pd.read_parquet(CFG.path / f"valid_data_{CFG.split_id}.parquet")
df_train = df_train.merge(bpp_df, on="sequence_id", how="left").reset_index(drop=True)
df_valid = df_valid.merge(bpp_df, on="sequence_id", how="left").reset_index(drop=True)


ds_train = RNA_DatasetV1(df_train)
ds_train_len = RNA_DatasetV1(df_train, mask_only=True)
sampler_train = torch.utils.data.RandomSampler(ds_train_len)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=CFG.bs,
            drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=CFG.num_workers,
            persistent_workers=True), CFG.device)


ds_val = RNA_DatasetV1(df_valid)
ds_val_len = RNA_DatasetV1(df_valid,mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=True)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)
gc.collect()
data = DataLoaders(dl_train,dl_val)

In [6]:
#x, y  = next(iter(dl_train))

In [7]:
from timm.models.layers import drop_path, to_2tuple, trunc_normal_
from einops import rearrange, repeat, reduce, pack, unpack
from einops.layers.torch import Rearrange, Reduce
from typing import Union, Tuple

class DropPath(nn.Module):
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)
    
    def extra_repr(self) -> str:
        return 'p={}'.format(self.drop_prob)
    
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x
    
class RotaryEmbedding(nn.Module):
    def __init__(self, dim, scale_base = 512, use_xpos = True):
        super().__init__()
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq)

        self.use_xpos = use_xpos
        self.scale_base = scale_base
        scale = (torch.arange(0, dim, 2) + 0.4 * dim) / (1.4 * dim)
        self.register_buffer('scale', scale)

    def forward(self, seq_len, device='cuda'):
        t = torch.arange(seq_len, device = device).type_as(self.inv_freq)
        freqs = torch.einsum('i , j -> i j', t, self.inv_freq)
        freqs = torch.cat((freqs, freqs), dim = -1)

        if not self.use_xpos:
            return freqs, torch.ones(1, device = device)

        power = (t - (seq_len // 2)) / self.scale_base
        scale = self.scale ** rearrange(power, 'n -> n 1')
        scale = torch.cat((scale, scale), dim = -1)

        return freqs, scale

def rotate_half(x):
    x1, x2 = x.chunk(2, dim=-1)
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(pos, t, scale = 1.):
    return (t * pos.cos() * scale) + (rotate_half(t) * pos.sin() * scale)

class Conv1D(nn.Conv1d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.src_key_padding_mask = None

    def forward(self, x, src_key_padding_mask=None):
        if src_key_padding_mask is not None: 
            self.src_key_padding_mask = src_key_padding_mask
        if self.src_key_padding_mask is not None:
            x = torch.where(self.src_key_padding_mask.unsqueeze(-1
                    ).expand(-1,-1,x.shape[-1]).bool(), torch.zeros_like(x), x)
        return super().forward(x.permute(0,2,1)).permute(0,2,1)
    
class ResBlock(nn.Sequential):
    def __init__(self, d_model):
        super().__init__(nn.LayerNorm(d_model), nn.GELU(), 
                         Conv1D(d_model,d_model,3,padding=1))
        self.src_key_padding_mask = None
        
    def forward(self, x, src_key_padding_mask=None):
        self[-1].src_key_padding_mask = src_key_padding_mask if \
            src_key_padding_mask is not None else self.src_key_padding_mask
        return x + super().forward(x)
    
class Extractor(nn.Sequential):
    def __init__(self, d_model, in_ch=4):
        super().__init__(nn.Embedding(in_ch,d_model//4), 
                Conv1D(d_model//4,d_model,7,padding=3), 
                ResBlock(d_model))
        
    def forward(self, x, src_key_padding_mask=None):
        for i in [1,2]:
            self[i].src_key_padding_mask = src_key_padding_mask
        return super().forward(x)

#BEiTv2 block
class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., init_values=None, act_layer=nn.GELU, norm_layer=nn.LayerNorm,
                 window_size=None, attn_head_dim=None, **kwargs):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = nn.MultiheadAttention(dim, num_heads, dropout=drop, batch_first=True)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        if init_values is not None:
            self.gamma_1 = nn.Parameter(init_values * torch.ones((dim)),requires_grad=True)
            self.gamma_2 = nn.Parameter(init_values * torch.ones((dim)),requires_grad=True)
        else:
            self.gamma_1, self.gamma_2 = None, None
            
        self.emb = RotaryEmbedding(dim)

    def forward(self, x, attn_mask=None, key_padding_mask=None):
        q = k = v = self.norm1(x)
        positions, scale = self.emb(x.shape[1],x.device)
        q = apply_rotary_pos_emb(positions, q, scale)
        k = apply_rotary_pos_emb(positions, k, scale ** -1)
        
        if self.gamma_1 is None:
            x = x + self.drop_path(self.attn(q,k,v,
                            attn_mask=attn_mask,
                            key_padding_mask=key_padding_mask,
                            need_weights=False)[0])
            x = x + self.drop_path(self.mlp(self.norm2(x)))
        else:
            x = x + self.drop_path(self.gamma_1 * self.attn(q,k,v,
                            attn_mask=attn_mask,
                            key_padding_mask=key_padding_mask,
                            need_weights=False)[0])
            x = x + self.drop_path(self.gamma_2 * self.mlp(self.norm2(x)))
        return x
    
class Block_conv(Block):
    def __init__(self, dim, mlp_ratio, *args, **kwargs):
        super().__init__(dim, *args, **kwargs)
        self.mlp.fc1= Conv1D(dim,dim,3,padding=1)
        self.mlp.fc2 = Conv1D(dim,dim,3,padding=1)

    def forward(self, *args, key_padding_mask=None, **kwargs):
        self.mlp.fc1.src_key_padding_mask = key_padding_mask
        self.mlp.fc2.src_key_padding_mask = key_padding_mask
        return super().forward(*args, **kwargs)
    
class RNA_Model(nn.Module):
    def __init__(self, dim=192, depth=12, head_size=32, **kwargs):
        super().__init__()
        #self.emb = nn.Sequential(nn.Embedding(4,dim//4), Conv1D(dim//4,dim,7,padding=3),
        #                        nn.LayerNorm(dim), nn.GELU(), Conv1D(dim,dim,3,padding=1))
        self.extractor = Extractor(dim)
        #self.pos_enc = SinusoidalPosEmb(dim)
        
        self.blocks = nn.ModuleList([ 
            Block_conv(
                dim=dim, num_heads=dim//head_size, mlp_ratio=4, drop_path=0.2*(i/(depth-1)), init_values=1,
                drop=0.1)
            for i in range(depth)])
        
        #self.transformer = nn.TransformerEncoder(
        #    TransformerEncoderLayer_conv(d_model=dim, nhead=dim//head_size, dim_feedforward=4*dim,
        #        dropout=0.1, activation=nn.GELU(), batch_first=True, norm_first=True), depth)
        self.proj_out = nn.Linear(dim,2)
    
    def forward(self, x0):
        mask = x0['mask']
        L0 = mask.shape[1]
        Lmax = mask.sum(-1).max()
        mask = mask[:,:Lmax]
        x = x0['seq'][:,:Lmax]
        
        #pos = torch.arange(Lmax, device=x.device).unsqueeze(0)
        #pos = self.pos_enc(pos)
        #self.emb[1].src_key_padding_mask=~mask
        #self.emb[4].src_key_padding_mask=~mask
        #x = self.emb(x)
        x = self.extractor(x, src_key_padding_mask=~mask)
        #x = x + pos
        
        #x = self.transformer(x, src_key_padding_mask=~mask)
        
        for blk in self.blocks:
            x = blk(x,key_padding_mask=~mask)   
        x = self.proj_out(x)
        
        x = F.pad(x,(0,0,0,L0-Lmax,0,0))
        return x

In [14]:
learn = Learner(data,
                RNA_Model().cuda(), 
                path = CFG.out, 
                loss_func=loss_laplace,
                cbs=[GradientClip(3.0),
                       CSVLogger(),
                     SAM(rho=.01),
                    SaveModelCallback(monitor='mae',comp=np.less,every_epoch=True)],
                metrics=[MAE()]).to_fp16() 
learn.fit_one_cycle(16, lr_max=5e-4, wd=0.05, pct_start=0.02)
#learn.validate()

epoch,train_loss,valid_loss,mae,time
0,1.180769,1.330352,0.508791,26:55
1,1.024785,1.145629,0.508791,26:57
2,0.909499,1.004368,0.508790,26:58
3,0.820862,0.907868,0.508432,26:59
4,0.752007,0.832408,0.481439,26:59
5,0.699856,0.772221,0.447463,27:01
6,0.660005,0.731462,0.415771,26:52
7,0.629822,0.689449,0.380177,26:47
8,0.607132,0.666272,0.353707,26:47
9,0.591858,0.651014,0.341738,26:47
